Imports

In [18]:
import pandas as pd
import csv, json
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm

Import Datasets to convert to geo json files

In [5]:
light_rail_stations1 = pd.read_csv("LYNX_Blue_Line_Stations.csv")
light_rail_stations2 = pd.read_csv("LYNX_Gold_Line_Stops.csv")

Mergeing

In [6]:
light_rails = pd.concat([light_rail_stations1, light_rail_stations2], ignore_index=True)

Creating Function to Convert Adress Based Locations to Cord Based Locations

In [12]:
# Load stations
stations = light_rails
# Initialize geocoder
geolocator = Nominatim(user_agent="charlotte_light_rail_analysis")

latitudes = []
longitudes = []

for _, row in tqdm(stations.iterrows(), total=len(stations)):
    address = f"{row['Address']}, {["Charlotte"]}, {r"NC"}"
    try:
        location = geolocator.geocode(address)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
        time.sleep(1)  # REQUIRED: respect Nominatim rate limits
    except:
        latitudes.append(None)
        longitudes.append(None)

stations["latitude"] = latitudes
stations["longitude"] = longitudes

#stations.to_csv("light_rail_stations_with_coords.csv", index=False)

100%|██████████| 43/43 [01:44<00:00,  2.42s/it]


In [19]:
def dataframe_to_geojson(
    df,
    lat_col="latitude",
    lon_col="longitude",
    id_col=None
):
    # Make a safe copy
    df = df.copy()

    # Ensure numeric coordinates
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")

    # Drop rows with missing coords
    df = df.dropna(subset=[lat_col, lon_col])

    features = []

    for _, row in df.iterrows():
        lat = row[lat_col]
        lon = row[lon_col]

        # Build properties (everything except lat/lon)
        properties = {
            col: (None if pd.isna(row[col]) else row[col])
            for col in df.columns
            if col not in [lat_col, lon_col]
        }

        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [float(lon), float(lat)]
            },
            "properties": properties
        }

        if id_col and id_col in row:
            feature["id"] = row[id_col]

        features.append(feature)

    return {
        "type": "FeatureCollection",
        "features": features
    }


In [14]:
geojson_data = dataframe_to_geojson(
    stations,
    lat_col="latitude",
    lon_col="longitude"
)

with open("output.geojson", "w") as f:
    json.dump(geojson_data, f, indent=2)


Converting weather crime data to a geojson

In [16]:
crime_with_weather = pd.read_csv("crime_with_weather.csv")

In [20]:
geojson_data = dataframe_to_geojson(
    crime_with_weather,
    lat_col="LATITUDE_PUBLIC",
    lon_col="LONGITUDE_PUBLIC"
)

with open("output.geojson", "w") as f:
    json.dump(geojson_data, f, indent=2)
